# Dr LIM - Dimensionality reduction by Learning Invariant Mapping

- This paper (similar to TSNE) proposes an alternative method to achieve dimensionality reduction

In [11]:
from functools import reduce
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

## CNN used in the paper 

![CNN architecture](./images/cnn.png)

In [20]:
class DrlimCNN(nn.Module):
    def __init__(self):
        super(DrlimCNN, self).__init__()
        
        # Layer 1:
        # n_input_channel = 1
        # n_output_channel = 15
        # Kernel Size = 8 for padding = 0, stride = 1
        k_size = 6
        in_channels = 1
        out_channels = 15 
        self.layer_1 = nn.Conv2d(in_channels, out_channels, k_size)
        
        # Layer 2: Subsampling - Maxpooling
        # Kernel Size = 19  for padding=0 and stride = 1
        k_size = 19
        self.max_pooling = nn.MaxPool2d(k_size, stride=1)
        
        # Layer 3: Conv layer
        # n_input_channel = 15
        # n_output_channel = 30
        # Kernel size = 9
        in_channels = 15
        out_channels = 30
        k_size = 9
        self.layer_3 = nn.Conv2d(in_channels, out_channels, k_size)
        
        # Layer 4: Fully connected
        self.output_layer = nn.Linear(30, 2)
        
        # Output probs
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        # Add relu on top of conv layer
        x = F.relu(self.layer_1(x))
        
        # Maxpool 
        x = self.max_pooling(x)
        
        # Another conv
        x = self.layer_3(x)
        
        # Get the size except for batch
        print("Shae")
        print(x.shape)
        num_flat_features = reduce(lambda x, y: x * y, x.shape[1:])
        print(num_flat_features)
        
        # Flatten
        x = x.reshape(1, num_flat_features)
        
        # Fully connected 
        x = self.output_layer(x)
        x = self.softmax(x)
        
        return x
    
    
net = DrlimCNN()
print(net)

DrlimCNN(
  (layer_1): Conv2d(1, 15, kernel_size=(6, 6), stride=(1, 1))
  (max_pooling): MaxPool2d(kernel_size=19, stride=1, padding=0, dilation=1, ceil_mode=False)
  (layer_3): Conv2d(15, 30, kernel_size=(9, 9), stride=(1, 1))
  (output_layer): Linear(in_features=30, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)


Shae
torch.Size([1, 30, 1, 1])
30


tensor([[0.1966, 0.8034]], grad_fn=<SoftmaxBackward>)